This is going to be the Jupyter Notebook for the IBM Dataset

In [ ]:
import os
import tarfile
import urllib.request
import pandas as pd

In [ ]:
load_ibm_data = pd.read_csv("C:/Users/alt98/Desktop/Machine Learning Project/Employee_Retention/Dataset - IBM.csv")

In [ ]:
ibm_data = load_ibm_data.copy()
#make sure it works
ibm_data.columns

In [ ]:
#change titles to lowercase
ibm_data.columns = ibm_data.columns.str.lower()

In [ ]:
#let's see what kind of data we have here
ibm_data.info()
# o null values in the dataset, so we can proceed with the analysis

In [ ]:
# Get value counts for all category-type columns
categorical_columns = ibm_data.select_dtypes(include='object')  # Select category-type columns
value_counts = {col: categorical_columns[col].value_counts() for col in categorical_columns.columns}

# Print the value counts for each column
for col, counts in value_counts.items():
    print(f"Value counts for column '{col}':")
    print(counts)
    print()

In [ ]:
#look numerical columns
ibm_data.describe()


In [ ]:
# Visualize the numeric columns
%matplotlib inline
import matplotlib.pyplot as plt
ibm_data.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
#drop the columns that are not needed for the analysis
ibm_data.drop(columns=['employeenumber', 'standardhours'], inplace=True)

In [ ]:
#create a training set
import numpy as np

In [ ]:
#compute a hash of each instance identifier 
from zlib import crc32
def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]


In [ ]:
#create an identifier column using the row index
ibm_data_with_id = ibm_data.reset_index()
train_set, test_set =  split_train_test_by_id(ibm_data_with_id, 0.2, "index")

In [ ]:
# We are looking to measure attrition so let's visualize the ratio of yes to no.
ibm_data['attrition'].hist()
plt.show()
#the ratio of yes to know is adequate for the exploratory analysis.

In [ ]:
#set up stratified sampling
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(ibm_data, ibm_data['attrition']):
    strat_train_set = ibm_data.loc[train_index]
    strat_test_set = ibm_data.loc[test_index]
#check the stratified sampling
strat_test_set['attrition'].value_counts() 


In [ ]:
#change categorical values to dummy variables
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
categorical_columns_1hot = cat_encoder.fit_transform(categorical_columns)
categorical_columns_1hot 

In [ ]:
#change encoded columns to float64
categorical_columns_1hot = categorical_columns_1hot.astype('int64')

In [26]:
# Encode the 'attrition' column to numeric values in the original dataframe
ibm_data['attrition'] = ibm_data['attrition'].map({'Yes': 1, 'No': 0})

# Create a correlation matrix using only numeric columns
numeric_ibm_data = ibm_data.select_dtypes(include=['number'])
corr_matrix = numeric_ibm_data.corr()

# Sort by correlation with 'attrition'
# Convert the one-hot encoded sparse matrix to a dense DataFrame
categorical_columns_1hot_df = pd.DataFrame(categorical_columns_1hot.toarray(), 
                                           columns=cat_encoder.get_feature_names_out(categorical_columns.columns),
                                           index=ibm_data.index)

# Concatenate the one-hot encoded columns with the original dataframe
ibm_data_encoded = pd.concat([numeric_ibm_data, categorical_columns_1hot_df], axis=1)

# Create a correlation matrix using the updated dataframe
corr_matrix = ibm_data_encoded.corr()

# Sort by correlation with 'attrition'
corr_matrix['attrition'].sort_values(ascending=False)

age                         NaN
attrition                   NaN
dailyrate                   NaN
distancefromhome            NaN
hourlyrate                  NaN
                             ..
stockoptionlevel_nan        NaN
worklifebalance_Bad         NaN
worklifebalance_Good        NaN
worklifebalance_Very Bad    NaN
worklifebalance_Very Good   NaN
Name: attrition, Length: 81, dtype: float64